# Base Span Detection implementation

1. Import dependencies

In [1]:
import random
import time
import torch
import os

import pandas as pd
import numpy as np

from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

from src.util.torch_device import resolve_torch_device
from src.data.span_detection_ds import ManipulationDetectionDataset
from src.definitions import (
    MODELS_FOLDER,
    RAW_DATA_FOLDER,
    SUBMISSIONS_FOLDER,
    PROCESSED_DATA_FOLDER,
)
from src.data.kaggle import submit_competition
from src.model.span_detection_metrics import compute_metrics

2. Prepare Env

In [2]:
random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

epoch_time = int(time.time())

os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

In [3]:
model_checkpoint = MODELS_FOLDER / "ua-fine-tuned-xlm-roberta-large"
result_model = f"span-detection-{str(model_checkpoint).split("/")[-1]}"

classifier_dropout = 0.1
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 5

submission_desc = (
    "Use xml-roberta-base fine tuned on a zeusfsx/ukrainian-news dataset as base model"
)

3. Load dataset

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

dataset_blueprint = ManipulationDetectionDataset(
    tokenizer=tokenizer,
    raw_path=RAW_DATA_FOLDER / "span-detection.parquet",
    processed_path=PROCESSED_DATA_FOLDER / "span-detection" / model_checkpoint,
    seed=random_seed,
    do_split=False
)

dataset = dataset_blueprint.read()

Saving the dataset (0/1 shards):   0%|          | 0/3822 [00:00<?, ? examples/s]

3. Prepare model

In [5]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(dataset_blueprint.label2id),
    id2label=dataset_blueprint.id2label,
    label2id=dataset_blueprint.label2id,
    classifier_dropout=classifier_dropout,
).to(device)

data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir=MODELS_FOLDER / f"{result_model}-checkpoint",
    learning_rate=learning_rate,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=random_seed,
    logging_steps=200,
    auto_find_batch_size=True,
    torch_empty_cache_steps=1000,
    metric_for_best_model="token_f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics(dataset_blueprint),
)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at /home/melal/Workspace/unlp-2025-manipulation-detector/models/ua-fine-tuned-xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4. Train model

In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss,Token F1,Token Precision,Token Recall,Span F1,Span Precision,Span Recall,Accuracy
1,0.464400,0.405721,0.438661,0.792727,0.303227,0.072843,0.068871,0.077301,0.798926
2,0.402500,0.358737,0.573100,0.818745,0.440837,0.101868,0.098555,0.105411,0.829836
3,0.364500,0.321859,0.649533,0.861855,0.521146,0.136481,0.139885,0.133240,0.854287
4,0.327800,0.243349,0.794592,0.817360,0.773058,0.200416,0.197676,0.203233,0.896443
5,0.292000,0.237709,0.790441,0.860607,0.730854,0.199430,0.202314,0.196627,0.899594


TrainOutput(global_step=1195, training_loss=0.35255520872491175, metrics={'train_runtime': 446.0585, 'train_samples_per_second': 42.842, 'train_steps_per_second': 2.679, 'total_flos': 4242022197230592.0, 'train_loss': 0.35255520872491175, 'epoch': 5.0})

5. Save weights

In [7]:
trainer.save_model(MODELS_FOLDER / result_model)
tokenizer.save_pretrained(MODELS_FOLDER / result_model)

('/home/melal/Workspace/unlp-2025-manipulation-detector/models/span-detection-ua-fine-tuned-xlm-roberta-large/tokenizer_config.json',
 '/home/melal/Workspace/unlp-2025-manipulation-detector/models/span-detection-ua-fine-tuned-xlm-roberta-large/special_tokens_map.json',
 '/home/melal/Workspace/unlp-2025-manipulation-detector/models/span-detection-ua-fine-tuned-xlm-roberta-large/tokenizer.json')

6. Test

In [8]:
model = AutoModelForTokenClassification.from_pretrained(MODELS_FOLDER / result_model)
tokenizer = AutoTokenizer.from_pretrained(MODELS_FOLDER / result_model)

In [9]:
submission_df = pd.read_csv(RAW_DATA_FOLDER / "test.csv")

In [10]:
nlp = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
)

Device set to use cuda:0


In [11]:
test_spans = {}
for i, row in submission_df.iterrows():
    res = nlp(row["content"])
    res_manipulation = [r for r in res if r["entity_group"] == "MANIPULATION"]
    res_manipulation_spans = [(r["start"], r["end"]) for r in res_manipulation]
    test_spans[row["id"]] = res_manipulation_spans

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [12]:
test_spans_df = pd.DataFrame(test_spans.items(), columns=["id", "trigger_words"])

In [14]:
submission_path = SUBMISSIONS_FOLDER / "span-detection" / f"{result_model}.csv"

submission_path.parent.mkdir(parents=True, exist_ok=True)

test_spans_df.to_csv(submission_path, index=False)

7. Submit

In [15]:
submission_params_str = f"classifier_dropout = {classifier_dropout}, learning_rate = {learning_rate}, weight_decay = {weight_decay}, num_train_epochs = {num_train_epochs}"
message = f"[ {submission_params_str} ] {submission_desc}"

submit_competition(
    submission_path, message, "unlp-2025-shared-task-span-identification"
)

100%|██████████| 411k/411k [00:00<00:00, 445kB/s] 
